<a href="https://colab.research.google.com/github/neural-anarchist/AI_Text_Annotation/blob/main/rhet_recognizationai_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
Medieval Spanish/Catalan Text Annotation Model
==============================================

A production-ready system for annotating medieval Spanish and Catalan texts
with literary, rhetorical, and linguistic taxonomies.

Author: Gary Shen
Version: 1.0.0
License: MIT

Usage:

    annotator = MedievalTextAnnotator(api_key="your_gemini_api_key")
    result = annotator.annotate_text("Your medieval text here...")

    # Access results
    print(f"Found {result.total_annotations} annotations")
    xml_output = result.xml_output
    csv_data = result.csv_output
    json_data = result.json_output
"""

!pip install pandas google-generativeai backoff
import os
import re
import time
import json
import logging
import traceback
from typing import List, Dict, Optional, Any, Tuple
from dataclasses import dataclass, field
from collections import Counter
import xml.etree.ElementTree as ET
from xml.dom import minidom


# Required dependencies
try:
    import pandas as pd
    import google.generativeai as genai
    import backoff
except ImportError as e:
    raise ImportError(f"Missing required dependency: {e}. Please install with: pip install pandas google-generativeai backoff")

# =============================================================================
# CONFIGURATION AND CONSTANTS
# =============================================================================

# Medieval Spanish/Catalan Literary Taxonomy
TAXONOMY = {
    "Genre": ["sermon", "vision", "consolatory_treatise", "vita", "confession",
              "prayer_devotional", "disputation", "performance", "music", "parable",
              "sacraments", "didactics", "scripture", "autobiography", "dialogue",
              "exemplum", "epistle", "theater_performance", "poetry", "music_laic",
              "testament_will", "speculum", "relacion", "genealogy", "testimony",
              "disputation_laic", "pastoral"],

    "Rhetoric": ["captatio", "colloquialism", "pathos", "logos", "ethos", "allegory",
                 "ekphrasis", "metaphor", "sign_symbol", "exegesis", "parallelism",
                 "didactics_rhet", "invective", "amplification", "anaphora", "antithesis",
                 "apostrophe", "exclamation", "polyptoton", "hypophora", "orality_literacy"],

    "Lexis": ["place", "person", "name", "gender", "building", "family", "authority",
              "body", "soul", "material", "nature", "animal", "time", "age", "food",
              "festivity", "latin", "sins", "virtues", "vice"],

    "Verb_Functions": ["affirm", "negate", "question", "exhort", "narrate", "describe",
                       "command", "promise", "lament", "praise", "blame", "warn",
                       "supplicate", "advise", "instruct", "prophesy", "invoke", "confess"],

    "Notes": ["hkbtext", "hkbobs", "intertext", "classical_sources", "biblical_sources",
              "contemporary_sources", "metatextual"]
}

# Create validation sets
ALL_VALID_SUBTYPES = set()
for subtypes in TAXONOMY.values():
    ALL_VALID_SUBTYPES.update(subtypes)

# Common subtype corrections for LLM responses
SUBTYPE_CORRECTIONS = {
    "capitatio": "captatio",
    "capitio": "captatio",
    "person_name": "person",
    "person_man": "person",
    "person_woman": "person",
    "description": "describe",
    "affirmation": "affirm",
    "negation": "negate",
    "questioning": "question",
    "exhortation": "exhort",
    "narration": "narrate",
    "commanding": "command",
    "promising": "promise",
    "lamenting": "lament",
    "praising": "praise",
    "blaming": "blame",
    "warning": "warn",
    "supplication": "supplicate",
    "advising": "advise",
    "instruction": "instruct",
    "instructing": "instruct",
    "prophecy": "prophesy",
    "prophesying": "prophesy",
    "invocation": "invoke",
    "invoking": "invoke",
    "confession": "confess",
    "confessing": "confess"
}

# =============================================================================
# DATA STRUCTURES
# =============================================================================

@dataclass
class SpanAnnotation:
    """Represents a single text annotation with span information."""
    text: str
    start: int
    end: int
    type: str
    subtype: str
    confidence: float = 1.0

    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary format for JSON serialization."""
        return {
            "span": {
                "text": self.text,
                "start": self.start,
                "end": self.end
            },
            "type": self.type,
            "subtype": self.subtype,
            "confidence": self.confidence
        }

    def __str__(self) -> str:
        return f"'{self.text}' ({self.type}:{self.subtype}) [{self.start}:{self.end}]"

@dataclass
class ChunkAnnotation:
    """Container for all annotations in a text chunk."""
    chunk_index: int
    original_text: str
    annotations: List[SpanAnnotation]
    processing_time: float
    chunk_size: int = field(init=False)

    def __post_init__(self):
        self.chunk_size = len(self.original_text)

    def to_ml_format(self) -> List[Dict[str, Any]]:
        """Convert to ML training format."""
        return [ann.to_dict() for ann in self.annotations]

    @property
    def annotation_count(self) -> int:
        """Get the number of annotations in this chunk."""
        return len(self.annotations)

@dataclass
class AnnotationResult:
    """Complete annotation result with all output formats."""
    xml_output: str
    csv_output: pd.DataFrame
    json_output: Dict[str, Any]
    total_annotations: int
    processing_time: float
    chunk_count: int
    statistics: Dict[str, Any]
    chunks: List[ChunkAnnotation] = field(default_factory=list)

    def save_outputs(self, output_dir: str, base_filename: str = "annotations") -> Dict[str, str]:
        """Save all outputs to files and return file paths."""
        os.makedirs(output_dir, exist_ok=True)

        file_paths = {}

        # Save XML
        xml_path = os.path.join(output_dir, f"{base_filename}.xml")
        with open(xml_path, 'w', encoding='utf-8') as f:
            f.write(self.xml_output)
        file_paths['xml'] = xml_path

        # Save CSV
        csv_path = os.path.join(output_dir, f"{base_filename}.csv")
        self.csv_output.to_csv(csv_path, index=False, encoding='utf-8')
        file_paths['csv'] = csv_path

        # Save JSON
        json_path = os.path.join(output_dir, f"{base_filename}_ml_data.json")
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(self.json_output, f, ensure_ascii=False, indent=2)
        file_paths['json'] = json_path

        return file_paths

# =============================================================================
# TEXT PROCESSING UTILITIES
# =============================================================================

class TextProcessor:
    """Handles text cleaning and chunking operations."""

    @staticmethod
    def clean_extracted_text(text: str) -> str:
        """Comprehensive text cleaning for Medieval Spanish/Catalan texts."""
        if not text:
            return ""

        original_length = len(text)

        # OCR artifacts and modern additions
        ocr_patterns = [
            r'\bPage\s+\d+\b',
            r'\bFolio\s+\d+[rv]?\b',
            r'\[p\.\s*\d+\]',
            r'\b\d{1,3}\s+of\s+\d{1,3}\b',
            r'^\s*\d+\s*$',
            r'^\s*[IVXLCDM]+\s*$',
            r'\d{1,2}/\d{1,2}/\d{2,4}\s+\d{1,2}:\d{2}(?:\:\d{2})?\s*[AP]M',
            r'\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}',
            r'https?://\S+',
            r'www\.\S+',
            r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',
            r'\bOCRed\b',
            r'\bOCR\b',
            r'Scanned\s+by.*?Internet\s+Archive',
            r'Digitized\s+by.*?Google',
            r'Archive\.org',
            r'Publicaciones.*?Montoto',
            r'Reflexiones.*?antiguo',
            r'Transcription\s+of\s+.*?TERESA\s+DE\s+CARTAGENA',
            r'TERESA\s+DE\s+CARTAGENA',
            r'^\s*[-*•]\s*$',
            r'^\s*[_=\-]{3,}\s*$',
            r'\[.*?\](?:\s*\[.*?\])*',
            r'(?:^|\n)\s*(?:CHAPTER|CAPÍTULO|CAP\.)\s+[IVXLCDM\d]+\s*(?:\n|$)',
            r'(?:^|\n)\s*\d+\s*(?:\n|$)',
        ]

        # Apply cleaning patterns
        for pattern in ocr_patterns:
            text = re.sub(pattern, '', text, flags=re.IGNORECASE | re.MULTILINE)

        # Normalize whitespace and punctuation
        text = re.sub(r'\s+', ' ', text)
        text = re.sub(r'\n\s*\n+', '\n\n', text)
        text = re.sub(r'([.!?])([A-ZÁÉÍÓÚÑÇ])', r'\1 \2', text)
        text = re.sub(r'([,;:])([A-ZÁÉÍÓÚÑÇ])', r'\1 \2', text)

        # Fix Medieval Spanish/Catalan character issues
        char_fixes = [
            (r'([aeiouáéíóú])n([aeiouáéíóú])', r'\1ñ\2'),
            (r'([aeiouáéíóú])c([aeiouáéíóú])', r'\1ç\2'),
            (r'(\w)j(\w)', r'\1i\2'),
            (r'(\w)v(\w)', r'\1u\2'),
            (r'\bu(\w)', r'v\1'),
        ]

        for pattern, replacement in char_fixes:
            text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)

        # Clean up excessive punctuation
        text = re.sub(r'[.]{3,}', '...', text)
        text = re.sub(r'[,]{2,}', ',', text)
        text = re.sub(r'[;]{2,}', ';', text)
        text = re.sub(r'[!]{2,}', '!', text)
        text = re.sub(r'[?]{2,}', '?', text)

        # Final cleanup
        text = text.strip()
        text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)

        cleaned_length = len(text)
        removed_percentage = ((original_length - cleaned_length) / original_length) * 100 if original_length > 0 else 0

        logging.info(f"Text cleaned: {original_length} → {cleaned_length} chars ({removed_percentage:.1f}% removed)")

        return text

    @staticmethod
    def chunk_text_intelligently(text: str, max_chunk_size: int = 1500) -> List[str]:
        """Smart text chunking that respects sentence and paragraph boundaries."""
        if not text or len(text) <= max_chunk_size:
            return [text] if text else []

        chunks = []
        current_chunk = ""

        # Split by paragraphs first
        paragraphs = text.split('\n\n')

        for paragraph in paragraphs:
            paragraph = paragraph.strip()
            if not paragraph:
                continue

            # If paragraph is too long, split by sentences
            if len(paragraph) > max_chunk_size:
                sentences = re.split(r'(?<=[.!?])\s+', paragraph)

                for sentence in sentences:
                    sentence = sentence.strip()
                    if not sentence:
                        continue

                    # Check if adding sentence would exceed limit
                    if len(current_chunk) + len(sentence) + 2 > max_chunk_size:
                        if current_chunk.strip():
                            chunks.append(current_chunk.strip())
                        current_chunk = sentence
                    else:
                        current_chunk += (" " + sentence) if current_chunk else sentence
            else:
                # Check if adding paragraph would exceed limit
                if len(current_chunk) + len(paragraph) + 2 > max_chunk_size:
                    if current_chunk.strip():
                        chunks.append(current_chunk.strip())
                    current_chunk = paragraph
                else:
                    current_chunk += ("\n\n" + paragraph) if current_chunk else paragraph

        # Add final chunk
        if current_chunk.strip():
            chunks.append(current_chunk.strip())

        # Filter out very short chunks
        chunks = [chunk for chunk in chunks if len(chunk) >= 50]

        logging.info(f"Created {len(chunks)} intelligent chunks")
        return chunks

# =============================================================================
# LLM INTEGRATION
# =============================================================================

class LLMAnnotator:
    """Handles interaction with Gemini API for annotation."""

    def __init__(self, api_key: str, model_name: str = 'gemini-1.5-flash'):
        """Initialize LLM annotator with API key."""
        self.api_key = api_key
        self.model_name = model_name
        self._configure_api()

    def _configure_api(self):
        """Configure Gemini API."""
        try:
            genai.configure(api_key=self.api_key)
            self.model = genai.GenerativeModel(self.model_name)
            logging.info(f"Gemini API configured with model: {self.model_name}")
        except Exception as e:
            raise ValueError(f"Failed to configure Gemini API: {e}")

    def _create_annotation_prompt(self, text_chunk: str) -> str:
        """Create comprehensive annotation prompt."""
        taxonomy_desc = ""
        for category, subtypes in TAXONOMY.items():
            taxonomy_desc += f"\n{category}:\n"
            for subtype in subtypes[:8]:  # Limit to avoid token overflow
                taxonomy_desc += f"  - {subtype}\n"
            if len(subtypes) > 8:
                taxonomy_desc += f"  ... and {len(subtypes) - 8} more\n"

        prompt = f"""You are an expert in Medieval Spanish and Catalan literature. Analyze the following text and identify meaningful spans that fit these categories:

{taxonomy_desc}

ANNOTATION RULES:
1. Only use the exact subtypes listed above
2. Identify spans of 2-50 words that clearly represent each category
3. Focus on significant literary, rhetorical, and semantic elements
4. Spans should not overlap
5. Return ONLY valid JSON array format
6. Be precise with character positions

EXAMPLES:
- "Teresa de Cartagena" → {{"span": {{"text": "Teresa de Cartagena", "start": 0, "end": 18}}, "type": "Lexis", "subtype": "person"}}
- "en el año de 1425" → {{"span": {{"text": "año de 1425", "start": 6, "end": 17}}, "type": "Lexis", "subtype": "time"}}
- "Dios todopoderoso" → {{"span": {{"text": "Dios todopoderoso", "start": 0, "end": 17}}, "type": "Lexis", "subtype": "authority"}}

TEXT TO ANALYZE ({len(text_chunk)} characters):
{text_chunk}

Return JSON array with this exact format:
[{{"span": {{"text": "exact_text_from_above", "start": start_index, "end": end_index}}, "type": "Category", "subtype": "exact_subtype"}}]

JSON:"""

        return prompt

    @backoff.on_exception(backoff.expo, Exception, max_tries=3, max_time=60)
    def _call_gemini_api(self, prompt: str) -> str:
        """Call Gemini API with retry logic."""
        try:
            response = self.model.generate_content(prompt)
            return response.text if response.text else ""
        except Exception as e:
            logging.error(f"Gemini API error: {e}")
            raise

    def _correct_subtype(self, subtype: str) -> str:
        """Correct common subtype errors."""
        subtype_lower = subtype.lower()
        return SUBTYPE_CORRECTIONS.get(subtype_lower, subtype)

    def _validate_annotation(self, annotation: Dict, original_text: str) -> Optional[SpanAnnotation]:
        """Validate and clean a single annotation."""
        try:
            # Extract span information
            span = annotation.get('span', {})
            if not isinstance(span, dict):
                return None

            text = span.get('text', '')
            start = span.get('start')
            end = span.get('end')
            ann_type = annotation.get('type', '')
            subtype = annotation.get('subtype', '')

            # Validate required fields
            if not all([text, isinstance(start, int), isinstance(end, int), ann_type, subtype]):
                return None

            # Validate indices
            if start < 0 or end > len(original_text) or start >= end:
                return None

            # Validate type exists in taxonomy
            if ann_type not in TAXONOMY:
                return None

            # Correct and validate subtype
            subtype = self._correct_subtype(subtype)
            if subtype not in TAXONOMY[ann_type]:
                return None

            # Validate text matches
            expected_text = original_text[start:end]
            if text.strip() != expected_text.strip():
                # Try to find the text in the original
                text_index = original_text.find(text.strip())
                if text_index != -1:
                    start = text_index
                    end = text_index + len(text.strip())
                else:
                    return None

            return SpanAnnotation(
                text=text.strip(),
                start=start,
                end=end,
                type=ann_type,
                subtype=subtype
            )

        except Exception as e:
            logging.debug(f"Error validating annotation: {e}")
            return None

    def _parse_gemini_response(self, response_text: str, original_text: str) -> List[SpanAnnotation]:
        """Parse Gemini response and extract valid annotations."""
        if not response_text:
            return []

        # Extract JSON from response
        json_patterns = [
            r'\[[\s\S]*?\]',
            r'```json\s*(\[[\s\S]*?\])\s*```',
            r'json\s*(\[[\s\S]*?\])',
        ]

        json_text = None
        for pattern in json_patterns:
            match = re.search(pattern, response_text, re.DOTALL)
            if match:
                json_text = match.group(1) if match.groups() else match.group(0)
                break

        if not json_text:
            logging.debug("No JSON found in LLM response")
            return []

        try:
            # Clean and parse JSON
            json_text = json_text.strip()
            if not json_text.startswith('['):
                json_text = '[' + json_text
            if not json_text.endswith(']'):
                json_text = json_text + ']'

            annotations_data = json.loads(json_text)
            if not isinstance(annotations_data, list):
                return []

            # Validate each annotation
            valid_annotations = []
            for ann_data in annotations_data:
                validated = self._validate_annotation(ann_data, original_text)
                if validated:
                    valid_annotations.append(validated)

            return valid_annotations

        except json.JSONDecodeError as e:
            logging.debug(f"JSON parsing error: {e}")
            return []
        except Exception as e:
            logging.debug(f"Unexpected error parsing response: {e}")
            return []

    def annotate_chunk(self, chunk: str, chunk_index: int) -> ChunkAnnotation:
        """Annotate a single text chunk."""
        start_time = time.time()

        try:
            # Create prompt
            prompt = self._create_annotation_prompt(chunk)

            # Call API
            response = self._call_gemini_api(prompt)

            # Parse response
            annotations = self._parse_gemini_response(response, chunk)

            processing_time = time.time() - start_time

            logging.info(f"Chunk {chunk_index}: {len(annotations)} annotations in {processing_time:.1f}s")

            return ChunkAnnotation(
                chunk_index=chunk_index,
                original_text=chunk,
                annotations=annotations,
                processing_time=processing_time
            )

        except Exception as e:
            logging.error(f"Error annotating chunk {chunk_index}: {e}")
            return ChunkAnnotation(
                chunk_index=chunk_index,
                original_text=chunk,
                annotations=[],
                processing_time=time.time() - start_time
            )

# =============================================================================
# OUTPUT GENERATORS
# =============================================================================

class OutputGenerator:
    """Handles generation of XML, CSV, and JSON outputs."""

    @staticmethod
    def generate_xml_output(chunk_annotations: List[ChunkAnnotation],
                          source_info: str = "user_input") -> str:
        """Generate XML output with embedded annotations."""
        # Create root element
        root = ET.Element('annotated_text')
        root.set('created', time.strftime('%Y-%m-%d %H:%M:%S'))
        root.set('total_chunks', str(len(chunk_annotations)))

        # Add metadata
        metadata = ET.SubElement(root, 'metadata')

        # Source info
        source_elem = ET.SubElement(metadata, 'source')
        source_elem.text = source_info

        # Add taxonomy
        taxonomy_elem = ET.SubElement(metadata, 'taxonomy')
        for category, subtypes in TAXONOMY.items():
            cat_elem = ET.SubElement(taxonomy_elem, 'category')
            cat_elem.set('name', category)
            for subtype in subtypes:
                sub_elem = ET.SubElement(cat_elem, 'subtype')
                sub_elem.text = subtype

        # Add annotated chunks
        body = ET.SubElement(root, 'body')
        total_annotations = 0

        for chunk_annotation in chunk_annotations:
            chunk_elem = ET.SubElement(body, 'chunk')
            chunk_elem.set('index', str(chunk_annotation.chunk_index))
            chunk_elem.set('length', str(chunk_annotation.chunk_size))
            chunk_elem.set('annotations', str(chunk_annotation.annotation_count))
            chunk_elem.set('processing_time', f"{chunk_annotation.processing_time:.2f}")

            if chunk_annotation.annotations:
                # Sort annotations by start position
                sorted_annotations = sorted(chunk_annotation.annotations, key=lambda x: x.start)

                # Build annotated text
                text_elem = ET.SubElement(chunk_elem, 'text')
                last_end = 0

                for ann in sorted_annotations:
                    # Add plain text before annotation
                    if ann.start > last_end:
                        plain_text = chunk_annotation.original_text[last_end:ann.start]
                        if plain_text.strip():
                            plain_elem = ET.SubElement(text_elem, 'plain')
                            plain_elem.text = plain_text

                    # Add annotated segment
                    seg_elem = ET.SubElement(text_elem, 'seg')
                    seg_elem.set('type', ann.type)
                    seg_elem.set('subtype', ann.subtype)
                    seg_elem.set('start', str(ann.start))
                    seg_elem.set('end', str(ann.end))
                    seg_elem.text = ann.text

                    last_end = ann.end

                # Add remaining plain text
                if last_end < len(chunk_annotation.original_text):
                    remaining_text = chunk_annotation.original_text[last_end:]
                    if remaining_text.strip():
                        plain_elem = ET.SubElement(text_elem, 'plain')
                        plain_elem.text = remaining_text

                total_annotations += len(chunk_annotation.annotations)
            else:
                # No annotations, just add original text
                text_elem = ET.SubElement(chunk_elem, 'text')
                plain_elem = ET.SubElement(text_elem, 'plain')
                plain_elem.text = chunk_annotation.original_text

        # Add summary statistics
        summary = ET.SubElement(metadata, 'summary')
        summary.set('total_annotations', str(total_annotations))

        # Calculate statistics
        type_counts = Counter()
        subtype_counts = Counter()

        for chunk in chunk_annotations:
            for ann in chunk.annotations:
                type_counts[ann.type] += 1
                subtype_counts[f"{ann.type}:{ann.subtype}"] += 1

        # Add type statistics
        if type_counts:
            types_elem = ET.SubElement(summary, 'types')
            for type_name, count in type_counts.most_common():
                type_stat = ET.SubElement(types_elem, 'type')
                type_stat.set('name', type_name)
                type_stat.set('count', str(count))

        # Add subtype statistics
        if subtype_counts:
            subtypes_elem = ET.SubElement(summary, 'subtypes')
            for subtype_name, count in subtype_counts.most_common():
                subtype_stat = ET.SubElement(subtypes_elem, 'subtype')
                subtype_stat.set('name', subtype_name)
                subtype_stat.set('count', str(count))

        # Convert to pretty XML
        rough_string = ET.tostring(root, encoding='unicode')
        reparsed = minidom.parseString(rough_string)
        return reparsed.toprettyxml(indent='  ')

    @staticmethod
    def generate_csv_output(chunk_annotations: List[ChunkAnnotation],
                          source_info: str = "user_input") -> pd.DataFrame:
        """Generate CSV output with annotation data."""
        rows = []

        for chunk_annotation in chunk_annotations:
            for ann in chunk_annotation.annotations:
                rows.append({
                    'chunk_index': chunk_annotation.chunk_index,
                    'annotation_text': ann.text,
                    'start_position': ann.start,
                    'end_position': ann.end,
                    'type': ann.type,
                    'subtype': ann.subtype,
                    'text_length': len(ann.text),
                    'chunk_length': chunk_annotation.chunk_size,
                    'processing_time': chunk_annotation.processing_time,
                    'confidence': ann.confidence,
                    'source': source_info
                })

        return pd.DataFrame(rows) if rows else pd.DataFrame()

    @staticmethod
    def generate_json_output(chunk_annotations: List[ChunkAnnotation],
                           source_info: str = "user_input") -> Dict[str, Any]:
        """Generate JSON output for ML training."""
        output = {
            "metadata": {
                "created": time.strftime('%Y-%m-%d %H:%M:%S'),
                "total_chunks": len(chunk_annotations),
                "source": source_info,
                "taxonomy": TAXONOMY
            },
            "annotations": []
        }

        total_annotations = 0

        for chunk_annotation in chunk_annotations:
            chunk_data = {
                "chunk_index": chunk_annotation.chunk_index,
                "original_text": chunk_annotation.original_text,
                "text_length": chunk_annotation.chunk_size,
                "processing_time": chunk_annotation.processing_time,
                "annotations": chunk_annotation.to_ml_format()
            }

            output["annotations"].append(chunk_data)
            total_annotations += len(chunk_annotation.annotations)

        # Add summary statistics
        output["metadata"]["total_annotations"] = total_annotations

        # Calculate statistics
        type_counts = Counter()
        subtype_counts = Counter()

        for chunk in chunk_annotations:
            for ann in chunk.annotations:
                type_counts[ann.type] += 1
                subtype_counts[f"{ann.type}:{ann.subtype}"] += 1

        output["metadata"]["statistics"] = {
            "type_counts": dict(type_counts),
            "subtype_counts": dict(subtype_counts)
        }

        return output


In [3]:
# =============================================================================
# MAIN ANNOTATOR CLASS
# =============================================================================

class MedievalTextAnnotator:
    """
    Main annotation class for Medieval Spanish/Catalan texts.

    This class provides a simple interface for annotating texts with
    medieval literary taxonomy, producing outputs in XML, CSV, and JSON formats.
    """

    def __init__(self, api_key: str, model_name: str = 'gemini-1.5-flash'):
        """
        Initialize the Medieval Text Annotator.

        Args:
            api_key: Your Gemini API key
            model_name: Gemini model to use (default: 'gemini-1.5-flash')
        """
        self.api_key = api_key
        self.model_name = model_name

        # Initialize components
        self.text_processor = TextProcessor()
        self.llm_annotator = LLMAnnotator(api_key, model_name)
        self.output_generator = OutputGenerator()

        # Set up logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

        logging.info("Medieval Text Annotator initialized successfully")

    def annotate_text(self, text: str, max_chunk_size: int = 1500) -> AnnotationResult:
        """
        Annotate Spanish/Catalan text with medieval literary taxonomy.

        Args:
            text: Raw text to annotate
            max_chunk_size: Maximum characters per chunk for processing

        Returns:
            AnnotationResult containing all three output formats
        """
        if not text or not text.strip():
            return self._create_empty_result()

        start_time = time.time()

        try:
            # Step 1: Clean the input text
            logging.info("Starting text annotation process...")
            cleaned_text = self.text_processor.clean_extracted_text(text)

            if not cleaned_text:
                logging.warning("Text cleaning resulted in empty text")
                return self._create_empty_result()

            # Step 2: Chunk the text intelligently
            chunks = self.text_processor.chunk_text_intelligently(cleaned_text, max_chunk_size)

            if not chunks:
                logging.warning("Text chunking resulted in no chunks")
                return self._create_empty_result()

            # Step 3: Annotate each chunk
            chunk_annotations = []

            for i, chunk in enumerate(chunks):
                logging.info(f"Processing chunk {i+1}/{len(chunks)}...")
                chunk_annotation = self.llm_annotator.annotate_chunk(chunk, i)
                chunk_annotations.append(chunk_annotation)

                # Small delay to avoid rate limiting
                if i < len(chunks) - 1:
                    time.sleep(0.5)

            # Step 4: Generate all output formats
            total_processing_time = time.time() - start_time

            xml_output = self.output_generator.generate_xml_output(chunk_annotations)
            csv_output = self.output_generator.generate_csv_output(chunk_annotations)
            json_output = self.output_generator.generate_json_output(chunk_annotations)

            # Calculate statistics
            total_annotations = sum(len(chunk.annotations) for chunk in chunk_annotations)
            statistics = self._calculate_statistics(chunk_annotations)

            logging.info(f"Annotation complete: {total_annotations} annotations in {total_processing_time:.2f}s")

            return AnnotationResult(
                xml_output=xml_output,
                csv_output=csv_output,
                json_output=json_output,
                total_annotations=total_annotations,
                processing_time=total_processing_time,
                chunk_count=len(chunks),
                statistics=statistics,
                chunks=chunk_annotations
            )

        except Exception as e:
            logging.error(f"Error during annotation: {e}")
            logging.error(traceback.format_exc())
            return self._create_error_result(str(e))

    def annotate_batch(self, texts: List[str], max_chunk_size: int = 1500) -> List[AnnotationResult]:
        """
        Annotate multiple texts in batch.

        Args:
            texts: List of texts to annotate
            max_chunk_size: Maximum characters per chunk for processing

        Returns:
            List of AnnotationResult objects
        """
        results = []

        for i, text in enumerate(texts):
            logging.info(f"Processing text {i+1}/{len(texts)}")
            result = self.annotate_text(text, max_chunk_size)
            results.append(result)

            # Delay between texts to avoid rate limiting
            if i < len(texts) - 1:
                time.sleep(1.0)

        return results

    def get_taxonomy(self) -> Dict[str, List[str]]:
        """Get the current taxonomy structure."""
        return TAXONOMY.copy()

    def validate_text(self, text: str) -> Tuple[bool, str]:
        """
        Validate input text for annotation.

        Args:
            text: Text to validate

        Returns:
            Tuple of (is_valid, error_message)
        """
        if not text:
            return False, "Text is empty"

        if not isinstance(text, str):
            return False, "Text must be a string"

        if len(text.strip()) < 10:
            return False, "Text too short (minimum 10 characters)"

        if len(text) > 100000:
            return False, "Text too long (maximum 100,000 characters)"

        return True, "Text is valid"

    def get_xml_output(self, result: AnnotationResult) -> str:
        """
        Get XML output from annotation result.

        Args:
            result: AnnotationResult object

        Returns:
            XML string with embedded annotations
        """
        return result.xml_output

    def get_csv_output(self, result: AnnotationResult) -> pd.DataFrame:
        """
        Get CSV output from annotation result.

        Args:
            result: AnnotationResult object

        Returns:
            pandas DataFrame with annotation data
        """
        return result.csv_output

    def get_json_output(self, result: AnnotationResult) -> Dict[str, Any]:
        """
        Get JSON output from annotation result.

        Args:
            result: AnnotationResult object

        Returns:
            Dictionary with ML training format data
        """
        return result.json_output

    def save_results(self, result: AnnotationResult, output_dir: str,
                    base_filename: str = "annotations") -> Dict[str, str]:
        """
        Save annotation results to files.

        Args:
            result: AnnotationResult to save
            output_dir: Directory to save files
            base_filename: Base filename for outputs

        Returns:
            Dictionary mapping format to file path
        """
        return result.save_outputs(output_dir, base_filename)

    def get_statistics(self, result: AnnotationResult) -> Dict[str, Any]:
        """
        Get detailed statistics from annotation result.

        Args:
            result: AnnotationResult object

        Returns:
            Dictionary with annotation statistics
        """
        return result.statistics

    def _create_empty_result(self) -> AnnotationResult:
        """Create an empty annotation result."""
        empty_df = pd.DataFrame(columns=[
            'chunk_index', 'annotation_text', 'start_position', 'end_position',
            'type', 'subtype', 'text_length', 'chunk_length', 'processing_time',
            'confidence', 'source'
        ])

        return AnnotationResult(
            xml_output="<annotated_text><metadata><summary total_annotations='0'/></metadata><body></body></annotated_text>",
            csv_output=empty_df,
            json_output={"metadata": {"total_annotations": 0}, "annotations": []},
            total_annotations=0,
            processing_time=0.0,
            chunk_count=0,
            statistics={"type_counts": {}, "subtype_counts": {}},
            chunks=[]
        )

    def _create_error_result(self, error_message: str) -> AnnotationResult:
        """Create an error annotation result."""
        empty_df = pd.DataFrame(columns=[
            'chunk_index', 'annotation_text', 'start_position', 'end_position',
            'type', 'subtype', 'text_length', 'chunk_length', 'processing_time',
            'confidence', 'source'
        ])

        return AnnotationResult(
            xml_output=f"<annotated_text><error>{error_message}</error></annotated_text>",
            csv_output=empty_df,
            json_output={"error": error_message, "annotations": []},
            total_annotations=0,
            processing_time=0.0,
            chunk_count=0,
            statistics={"error": error_message},
            chunks=[]
        )

    def _calculate_statistics(self, chunk_annotations: List[ChunkAnnotation]) -> Dict[str, Any]:
        """Calculate detailed statistics for annotation results."""
        if not chunk_annotations:
            return {"type_counts": {}, "subtype_counts": {}}

        type_counts = Counter()
        subtype_counts = Counter()
        text_lengths = []
        processing_times = []

        for chunk in chunk_annotations:
            processing_times.append(chunk.processing_time)

            for ann in chunk.annotations:
                type_counts[ann.type] += 1
                subtype_counts[f"{ann.type}:{ann.subtype}"] += 1
                text_lengths.append(len(ann.text))

        statistics = {
            "type_counts": dict(type_counts),
            "subtype_counts": dict(subtype_counts),
            "avg_processing_time": sum(processing_times) / len(processing_times) if processing_times else 0,
            "total_processing_time": sum(processing_times),
            "avg_annotation_length": sum(text_lengths) / len(text_lengths) if text_lengths else 0,
            "annotation_density": len(text_lengths) / sum(chunk.chunk_size for chunk in chunk_annotations) if chunk_annotations else 0
        }

        return statistics